In [25]:
print("Hello")

Hello


In [26]:
import pandas as pd
import os


# Directory where your CSV files are located
folder_path = './summarized_patents/'


# List to hold data from each CSV file
data_frames = []


# Loop through all the CSV files in the directory
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV file and append it to the list of data frames
        df = pd.read_csv(file_path)
        data_frames.append(df)


# Concatenate all the data frames into one
merged_df = pd.concat(data_frames, ignore_index=True)


# Save the merged data frame to a new CSV file
output_file_path = './test_summarization_loop/merged_file.csv'
merged_df.to_csv(output_file_path, index=False)


print(f'Merged CSV saved to {output_file_path}')


Merged CSV saved to ./test_summarization_loop/merged_file.csv


In [27]:
import pandas as pd
data_patent = pd.read_csv('./test_summarization_loop/merged_file.csv')

In [29]:
rows = len(data_patent.axes[0])
cols = len(data_patent.axes[1])
print(rows, cols)

3720 11


In [30]:
patent_data = data_patent[["PatentNumber","ApplicationReference","Date","OrganizationName","SummaryofPatent"]]

In [31]:
rows = len(patent_data.axes[0])
cols = len(patent_data.axes[1])
print(rows, cols)

3720 5


In [32]:
patent_data.to_csv('./test_summarization_loop/patents_summarized_abstract_clean.csv', index=False)

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [2]:
loader = CSVLoader(file_path='./test_summarization_loop/patents_summarized_abstract_clean.csv')

documents = loader.load()

In [3]:
print(type(documents))

<class 'list'>


In [5]:
documents[0].page_content

'PatentNumber: 11925278\nApplicationReference: utility\nDate: 2024-03-12\nOrganizationName: SAMSUNG ELECTRONICS CO., LTD.\nSummaryofPatent: Summary:\nThe display apparatus aims to provide a flexible and adjustable mounting system for displays, allowing for rotation in multiple directions and slidable movement in the leftward and rightward directions. The connector includes rotatable supports and movable supports, which enable the mounting member to be securely attached to the fixing member while allowing for adjustment. The invention improves upon existing mounting systems by providing a more flexible and adjustable solution, but may be limited by the weight and size of the display. The energy efficiency of the invention is not explicitly mentioned in the provided text.'

In [4]:
len(documents)

3720

In [6]:
#Embedding model load

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    # ,
    # query_instruction="为这个句子生成表示以用于检索相关文章："
)


emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())

[codecarbon WARNING @ 17:56:13] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 17:56:13] [setup] RAM Tracking...
[codecarbon INFO @ 17:56:13] [setup] GPU Tracking...
[codecarbon INFO @ 17:56:13] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:56:13] [setup] CPU Tracking...
[codecarbon WARNING @ 17:56:13] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1716224173.7884898


[codecarbon WARNING @ 17:56:15] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:56:15] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 17:56:15] >>> Tracker's metadata:
[codecarbon INFO @ 17:56:15]   Platform system: Linux-5.4.0-174-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 17:56:15]   Python version: 3.12.1
[codecarbon INFO @ 17:56:15]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 17:56:15]   Available RAM : 503.339 GB
[codecarbon INFO @ 17:56:15]   CPU count: 112
[codecarbon INFO @ 17:56:15]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 17:56:15]   GPU count: 2
[codecarbon INFO @ 17:56:15]   GPU model: 2 x NVIDIA A40
/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/u

Emissions for this code execution is:  0.00014864047175184566
End of code time:  1716224183.7095902


In [8]:
print(model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-large-en-v1.5' cache_folder=None model_kwargs={} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction=''


In [9]:
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

In [42]:
 # Clear out the existing database directory if it exists
if os.path.exists("./db"):
    shutil.rmtree("./db")


In [10]:
#Chroma DB saving files and embeddings

from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

chroma_db = Chroma.from_documents(
    documents, model, persist_directory="./db"
)

chroma_db.persist()

emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())


[codecarbon WARNING @ 17:56:45] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 17:56:45] [setup] RAM Tracking...
[codecarbon INFO @ 17:56:45] [setup] GPU Tracking...
[codecarbon INFO @ 17:56:45] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:56:45] [setup] CPU Tracking...
[codecarbon WARNING @ 17:56:45] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1716224205.5088346


[codecarbon WARNING @ 17:56:46] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:56:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 17:56:46] >>> Tracker's metadata:
[codecarbon INFO @ 17:56:46]   Platform system: Linux-5.4.0-174-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 17:56:46]   Python version: 3.12.1
[codecarbon INFO @ 17:56:46]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 17:56:46]   Available RAM : 503.339 GB
[codecarbon INFO @ 17:56:46]   CPU count: 112
[codecarbon INFO @ 17:56:46]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 17:56:46]   GPU count: 2
[codecarbon INFO @ 17:56:46]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 17:57:04] Energy consumed for RAM : 0.000787 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 17:57:04] Energy consumed for all GPUs : 0.001460 kWh. Total GPU Power : 349.876985650

Emissions for this code execution is:  0.0014553289802961846
End of code time:  1716224245.5109322


In [11]:
vectordb = Chroma(persist_directory="./db", embedding_function= model)
retriever = vectordb.as_retriever(search_kwargs = {"k": 10})
print(retriever.search_type, retriever.search_kwargs)

similarity {'k': 10}


similarity {'k': 10}


In [27]:
retirve_result = retriever.get_relevant_documents("Sustainable and Environmentally Friendly RAM Memory for Mobile Phones")

In [28]:
len(retirve_result)

10

In [29]:
retirve_result[0]

Document(page_content='PatentNumber: 11961564\nApplicationReference: utility\nDate: 2024-04-16\nOrganizationName: Samsung Electronics Co., Ltd.\nSummaryofPatent: Summary:\nThe research aims to improve nonvolatile memory devices by vertically connecting cell regions and peripheral regions using first metal pads and second metal pads, respectively. The memory block includes multiple sub-blocks and cell strings connected in series, with intermediate switching transistors in the boundary between adjacent sub-blocks. During a program operation, the intermediate switching transistors are selectively activated based on the program address, allowing for more efficient programming. While the research shows potential for improved performance, it also mentions potential drawbacks such as increased complexity and reduced scalability. Energy efficiency is not explicitly discussed in the provided text.', metadata={'row': 3206, 'source': './test_summarization_loop/patents_summarized_abstract_clean.cs

In [30]:
import re

pattern = re.compile(r"SummaryofPatent: Summary:\n(.*)", re.DOTALL)

match = pattern.search(retirve_result[0].page_content)

if match:
    print(match.group(1).strip())

The research aims to improve nonvolatile memory devices by vertically connecting cell regions and peripheral regions using first metal pads and second metal pads, respectively. The memory block includes multiple sub-blocks and cell strings connected in series, with intermediate switching transistors in the boundary between adjacent sub-blocks. During a program operation, the intermediate switching transistors are selectively activated based on the program address, allowing for more efficient programming. While the research shows potential for improved performance, it also mentions potential drawbacks such as increased complexity and reduced scalability. Energy efficiency is not explicitly discussed in the provided text.


In [31]:
retirve_result[0]

Document(page_content='PatentNumber: 11961564\nApplicationReference: utility\nDate: 2024-04-16\nOrganizationName: Samsung Electronics Co., Ltd.\nSummaryofPatent: Summary:\nThe research aims to improve nonvolatile memory devices by vertically connecting cell regions and peripheral regions using first metal pads and second metal pads, respectively. The memory block includes multiple sub-blocks and cell strings connected in series, with intermediate switching transistors in the boundary between adjacent sub-blocks. During a program operation, the intermediate switching transistors are selectively activated based on the program address, allowing for more efficient programming. While the research shows potential for improved performance, it also mentions potential drawbacks such as increased complexity and reduced scalability. Energy efficiency is not explicitly discussed in the provided text.', metadata={'row': 3206, 'source': './test_summarization_loop/patents_summarized_abstract_clean.cs

In [36]:
# filtered_results = [doc for score, doc in retirve_result['SummaryofPatent'] if score >= "0.7"]
# print(filtered_results)

In [37]:
type(retirve_result)

list

In [38]:
import re

for item in retirve_result:
    pattern = re.compile(r"SummaryofPatent: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)

    if match:
        print(match.group(1).strip())

The research aims to improve nonvolatile memory devices by vertically connecting cell regions and peripheral regions using first metal pads and second metal pads, respectively. The memory block includes multiple sub-blocks and cell strings connected in series, with intermediate switching transistors in the boundary between adjacent sub-blocks. During a program operation, the intermediate switching transistors are selectively activated based on the program address, allowing for more efficient programming. While the research shows potential for improved performance, it also mentions potential drawbacks such as increased complexity and reduced scalability. Energy efficiency is not explicitly discussed in the provided text.
The research aims to improve the energy efficiency of nonvolatile memory devices by enabling in-place updates of data stored in memory cells. The proposed approach uses a storage controller to perform a MAC operation on data stored in a memory cell, and sets the cell to

In [39]:
import re

# Assuming the Document class has an attribute 'page_content' that stores the content
class Document:
    def __init__(self, page_content):
        self.page_content = page_content

for item in retirve_result:
    # Regular expression to capture Patent Number and Summary
    pattern = re.compile(r"PatentNumber: (\d+)\n.*SummaryofPatent: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)  # Use dot notation to access the page_content attribute

    if match:
        patent_number = match.group(1).strip()
        summary = match.group(2).strip()
        print("Patent Number:", patent_number)
        print("Summary:", summary)


Patent Number: 11961564
Summary: The research aims to improve nonvolatile memory devices by vertically connecting cell regions and peripheral regions using first metal pads and second metal pads, respectively. The memory block includes multiple sub-blocks and cell strings connected in series, with intermediate switching transistors in the boundary between adjacent sub-blocks. During a program operation, the intermediate switching transistors are selectively activated based on the program address, allowing for more efficient programming. While the research shows potential for improved performance, it also mentions potential drawbacks such as increased complexity and reduced scalability. Energy efficiency is not explicitly discussed in the provided text.
Patent Number: 11861237
Summary: The research aims to improve the energy efficiency of nonvolatile memory devices by enabling in-place updates of data stored in memory cells. The proposed approach uses a storage controller to perform a M

In [40]:
from IPython.display import display, HTML

style = """
<style>
    .research-summary {
        border: 1px solid #ddd;
        border-radius: 8px;
        padding: 15px;
        margin: 10px 0;
        background-color: #f9f9f9;
        color: black;
    }
    .research-summary h3 {
        color: #336;
    }
    .research-summary p {
        text-indent: 20px;
    }
</style>
"""

import re

# Assuming the Document class has an attribute 'page_content' that stores the content
class Document:
    def __init__(self, page_content):
        self.page_content = page_content

for item in retirve_result:
    # Regular expression to capture Patent Number and Summary
    pattern = re.compile(r"PatentNumber: (\d+)\n.*SummaryofPatent: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)  # Use dot notation to access the page_content attribute

    if match:
        contents = ""
        patent_number = match.group(1).strip()
        summary = match.group(2).strip()
        # print("Patent Number:", patent_number)
        # print("Summary:", summary)
        contents += f"""
            <div class="research-summary">
                <h3>Patent Number:{patent_number}</h3>
                <p>{summary}</p>
            </div>
            """
        display(HTML(style + contents))


